In [1]:
import os
import csv
import sys
import random
import scipy
import numpy as np
import xgboost as xgb
# read in data


workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'
month_dict_inv = dict()
month_dict_inv[0] = '2015-01'
month_dict_inv[1] = '2015-02'
month_dict_inv[2] = '2015-03'
month_dict_inv[3] = '2015-04'
month_dict_inv[4] = '2015-05'
month_dict_inv[5] = '2015-06'
month_dict_inv[6] = '2015-07'
month_dict_inv[7] = '2015-08'
month_dict_inv[8] = '2015-09'
month_dict_inv[9] = '2015-10'
month_dict_inv[10] = '2015-11'
month_dict_inv[11] = '2015-12'
month_dict_inv[12] = '2016-01'
month_dict_inv[13] = '2016-02'
month_dict_inv[14] = '2016-03'
month_dict_inv[15] = '2016-04'
month_dict_inv[16] = '2016-05'
month_dict_inv[17] = '2016-06'

/home/irashadow/env_TensorFlow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
cust_2_labels_dict = dict()

pos_case_dict = dict()
neg_case_dict = dict()


model_eval = 23


train_write = open(workspace_path + '/train/model_val/cust_product_m'+str(model_eval)+'_xgb.train', 'w')

valid_write = open(workspace_path + '/train/model_val/cust_product_m'+str(model_eval)+'_xgb.valid', 'w')

check_line_count = 0   
#data 5[2016June] for model #18
#data 16[2016May] for model #13
#data 12[2016Jan] for model #20
#data 11[2015Dec] for model #3
#data 9[2015Oct] for model #3
#----------------------------------
#try data 12[2016Jan] for model #5
#----------------------------------


feat_file = open(workspace_path + '/feat/4_9_cust_date_nominal_prod_L4_expand.feat', 'r')

for line in feat_file:

    line = line.strip()
    check_line_count += 1

    if len(line.split('|')) < 6:

       print line 

    cust_id = line.split('|')[0]
    date_note = line.split('|')[1]
    label_tokens = line.split('|')[2]

    if True:

        get_label = np.zeros(30) 

        for label_item in label_tokens.split(','):

            if len(label_item) > 0:

                prod_id = int(label_item)
                get_label[prod_id] = 1

#-------
        if date_note == month_dict_inv[5]:

           for j in range(1,25):

               if j not in [1,2,10,11,12,21]: 

                  if get_label[j] == 1 and j == model_eval:

                     prt_line = str(1) + ' ' 
                     for fi in range(3,9):
                         if len(line.split('|')[fi]) > 0:
                            prt_line += line.split('|')[fi] + ' ' 

                     train_write.write(prt_line[:-1]+'\n') 

                  elif get_label[j] == 1 and j != model_eval:

                     prt_line = str(0) + ' ' 
                     for fi in range(3,9):
                         if len(line.split('|')[fi]) > 0:
                            prt_line += line.split('|')[fi] + ' ' 

                     train_write.write(prt_line[:-1]+'\n')   


#-------
        if date_note == month_dict_inv[16]:

           for j in range(1,25):

               if j not in [1,2,10,11,12,21]: 

                  if get_label[j] == 1 and j == model_eval:

                     prt_line = str(1) + ' ' 
                     for fi in range(3,9):
                         if len(line.split('|')[fi]) > 0:
                            prt_line += line.split('|')[fi] + ' ' 

                     valid_write.write(prt_line[:-1]+'\n') 

                  elif get_label[j] == 1 and j != model_eval:

                     prt_line = str(0) + ' ' 
                     for fi in range(3,9):
                         if len(line.split('|')[fi]) > 0:
                            prt_line += line.split('|')[fi] + ' ' 

                     valid_write.write(prt_line[:-1]+'\n') 



feat_file.close()           
 
train_write.close()  
valid_write.close()
print check_line_count    

436493


In [9]:
train_data_path = './train/model_val/cust_product_m'+str(model_eval)+'_xgb.train'

dtrain = xgb.DMatrix(train_data_path)

# specify parameters via map
param ={"objective" : "multi:softprob",    # multiclass classification 
              "num_class" : 2,    # number of classes 
              "eval_metric" : "mlogloss",    # evaluation metric 
              "nthread" : 6,   # number of threads to be used 
              "max_depth" : 16,    # maximum depth of tree 
              "eta" : 0.5,    # step size shrinkage 
              "subsample" : 0.7,    # part of data instances to grow tree 
              "colsample_bytree" : 0.7  # subsample ratio of columns when constructing each tree 
       }

num_round = 50
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtrain)

print preds.shape

(44055, 2)


In [10]:
preds_sort = np.argsort(preds, axis=1)
print preds_sort.shape
preds_top7 = np.fliplr(preds_sort)[:,:7]
print preds_top7

#----------------------------------------------------
dtrain_label = dtrain.get_label().astype(int)

print dtrain_label.shape[0]

label_train_dict = dict()
for i in range(0,dtrain_label.shape[0]):
#for i in range(0,100):
    
    top_product = dtrain_label.astype(int)[i]
    #print top_product
    
    if top_product not in label_train_dict:
       label_train_dict[top_product]  = 1
    else:
       label_train_dict[top_product]  += 1
    
print len(label_train_dict)    

(44055, 2)
[[0 1]
 [0 1]
 [1 0]
 ..., 
 [0 1]
 [0 1]
 [0 1]]
44055
2


In [11]:
print 'train distribution'

for item in label_train_dict:
    
    print 'label #' + str(item) + ' :%.2f'%(label_train_dict[item]/float(dtrain_label.shape[0])*100.0) 
    #print 'label #' + str(item)


train distribution
label #0 :81.32
label #1 :18.68


In [12]:
correct_count = 0
TP_count = 0
FP_count = 0
TN_count = 0
FN_count = 0

for i in range(0,len(dtrain_label)):
    
    if preds_top7[i,0] == dtrain_label[i]:

       correct_count += 1 
    
       if preds_top7[i,0] == 1 and dtrain_label[i] == 1:
            
          TP_count += 1  
    
       elif preds_top7[i,0] == 0 and dtrain_label[i] == 0:
            
          TN_count += 1  
    
    else:
        
       if preds_top7[i,0] == 1 and dtrain_label[i] == 0:
            
          FP_count += 1  
    
       elif preds_top7[i,0] == 0 and dtrain_label[i] == 1:
            
          FN_count += 1   
 

print 'accuracy:%.3f'%(correct_count/float(len(dtrain_label)))

print 'precision:%.3f'%(TP_count/float(TP_count+FP_count))
print 'recall:%.3f'%(TP_count/float(TP_count+FN_count))

print 'TP_count:'+str(TP_count)
print 'FP_count:'+str(FP_count)
print 'FN_count:'+str(FN_count)
print 'TN_count:'+str(TN_count)

accuracy:0.874
precision:0.719
recall:0.537
TP_count:4422
FP_count:1732
FN_count:3807
TN_count:34094


In [15]:
valid_data_path = './train/model_val/cust_product_m'+str(model_eval)+'_xgb.valid'
dvalid = xgb.DMatrix(valid_data_path)
dvalid_label = dvalid.get_label().astype(int)
preds = bst.predict(dvalid)



In [40]:
preds_sort = np.argsort(preds, axis=1)
print preds_sort.shape
preds_top7 = np.fliplr(preds_sort)[:,:15]
print preds_top7

label_submit_dict = dict()
for i in range(0,preds_sort.shape[0]):
#for i in range(0,100):
    
    top_product = preds_top7[i,0]
    
    if top_product not in label_submit_dict:
       label_submit_dict[top_product]  = 1
    else:
       label_submit_dict[top_product]  += 1
    

(37843, 2)
[[0 1]
 [0 1]
 [0 1]
 ..., 
 [0 1]
 [0 1]
 [0 1]]


In [41]:
print 'validation distribution'

for item in label_submit_dict:
    
    print 'label #' + str(item+1) + ' :%.2f'%(label_submit_dict[item]/float(preds_sort.shape[0])*100.0) 
    #print 'label #' + str(item)


validation distribution
label #1 :91.88
label #2 :8.12


In [42]:
correct_count = 0
TP_count = 0
FP_count = 0
TN_count = 0
FN_count = 0

for i in range(0,len(dvalid_label)):
    
    if preds_top7[i,0] == dvalid_label[i]:

       correct_count += 1 
    
       if preds_top7[i,0] == 1 and dvalid_label[i] == 1:
            
          TP_count += 1  
    
       elif preds_top7[i,0] == 0 and dvalid_label[i] == 0:
            
          TN_count += 1  
    
    else:
        
       if preds_top7[i,0] == 1 and dvalid_label[i] == 0:
            
          FP_count += 1  
    
       elif preds_top7[i,0] == 0 and dvalid_label[i] == 1:
            
          FN_count += 1   
 

print 'accuracy:%.3f'%(correct_count/float(len(dvalid_label)))

print 'precision:%.3f'%(TP_count/float(TP_count+FP_count))
print 'recall:%.3f'%(TP_count/float(TP_count+FN_count))

print 'TP_count:'+str(TP_count)
print 'FP_count:'+str(FP_count)
print 'FN_count:'+str(FN_count)
print 'TN_count:'+str(TN_count)

accuracy:0.846
precision:0.455
recall:0.252
TP_count:1397
FP_count:1675
FN_count:4144
TN_count:30627
